In [13]:
#imports to work with...
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.data import DataLoader
import torch
import torchvision
from torchvision import transforms
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

from cl_framework.continual_learning.metrics.metric_evaluator_incdec import MetricEvaluatorIncDec
from cl_framework.utilities.matrix_logger import IncDecLogger
from torchmetrics import Recall

In [14]:
subcategories_names = ['eating burger','eating cake','eating carrots','eating chips','eating doughnuts',
                       'eating hotdog','eating ice cream','eating spaghetti','eating watermelon','sucking lolly',
                       'tasting beer','tasting food','tasting wine','sipping cup','texting','talking on cell phone',
                       'looking at phone','smoking','smoking hookah','smoking pipe','sleeping','yawning','headbanging',
                       'headbutting','shaking head','scrubbing face','putting in contact lenses','putting on eyeliner',
                       'putting on foundation','putting on lipstick','putting on mascara','brushing hair','brushing teeth',
                       'braiding hair','combing hair','dyeing eyebrows','dyeing hair']
all_behaviors_dict = {
    'food': [
        'eating burger', 'eating cake', 'eating carrots', 'eating chips', 'eating doughnuts',
        'eating hotdog', 'eating ice cream', 'eating spaghetti', 'eating watermelon',
        'sucking lolly', 'tasting beer', 'tasting food', 'tasting wine', 'sipping cup'
    ],
    'phone': [
        'texting', 'talking on cell phone', 'looking at phone'
    ],
    'smoking': [
        'smoking', 'smoking hookah', 'smoking pipe'
    ],
    'fatigue': [
        'sleeping', 'yawning', 'headbanging', 'headbutting', 'shaking head'
    ],
    'selfcare': [
        'scrubbing face', 'putting in contact lenses', 'putting on eyeliner', 'putting on foundation',
        'putting on lipstick', 'putting on mascara', 'brushing hair', 'brushing teeth', 'braiding hair',
        'combing hair', 'dyeing eyebrows', 'dyeing hair'
    ]
}

In [15]:
# ap of subcategories
mean_path = 'mean_over_tasks/forg_ap_per_subcategory.out'
std_path = 'std_over_tasks/forg_ap_per_subcategory.out'

path_to_baseline = '../runs_trainings/from_checkpoint_sgd/no_freeze/data_substitution/statistics/'
baseline_data = np.loadtxt(os.path.join(path_to_baseline,mean_path),delimiter=',')

path_to_decremental = '../runs_trainings/from_checkpoint_sgd/no_freeze/decremental/statistics/'
decremental_data = np.loadtxt(os.path.join(path_to_decremental,mean_path),delimiter=',')

path_to_incdec = '../runs_trainings/from_checkpoint_sgd/no_freeze/incremental_decremental/4_4_lr/statistics/'
incdec_data = np.loadtxt(os.path.join(path_to_incdec,mean_path),delimiter=',')

path_to_joint = '../runs_trainings/from_checkpoint_sgd/no_freeze/joint_incremental/statistics/'
joint_data = np.loadtxt(os.path.join(path_to_joint,mean_path),delimiter=',')

In [16]:
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
pre_trained_path = os.path.join(output_path,'pre_trained')
if not os.path.exists(pre_trained_path):
    os.mkdir(pre_trained_path)
ouput_ap_path = os.path.join(pre_trained_path,'ap')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
no_freeze_path = os.path.join(ouput_ap_path,'no_freeze')
if not os.path.exists(no_freeze_path):
    os.mkdir(no_freeze_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    baseline_list = []
    decremental_list = []
    incdec_list = []
    joint_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        baseline_list.append(baseline_data[pos_subcat_in_data])
        decremental_list.append(decremental_data[pos_subcat_in_data])
        incdec_list.append(incdec_data[pos_subcat_in_data])
        joint_list.append(joint_data[pos_subcat_in_data])
        

        

    subcat_means = {
        'Baseline': [baseline_list],
        'Decremental': [decremental_list],
        'Incdec': [incdec_list],
        'Joint': [joint_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], 0, fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat AP')
    ax.set_title('Subcategories Forgetting AP - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(no_freeze_path + '/class_' + class_name + '.png')
    plt.close(fig)

COMPARISONS BETWEEN DIFFERENT METHODS

In [17]:
# Comparison between best results
# ap of subcategories on incdec
mean_path = 'mean_over_tasks/forg_ap_per_subcategory.out'
std_path = 'std_over_tasks/forg_ap_per_subcategory.out'


path_to_lwf = '../runs_trainings/from_checkpoint_sgd/lwf/incremental_decremental/statistics/'
lwf_data = np.loadtxt(os.path.join(path_to_lwf,mean_path),delimiter=',')

path_to_nofreeze = '../runs_trainings/from_checkpoint_sgd/no_freeze/incremental_decremental/4_4_lr/statistics/'
nofreeze_data = np.loadtxt(os.path.join(path_to_nofreeze,mean_path),delimiter=',')

path_to_fd = '../runs_trainings/from_checkpoint_sgd/fd/incremental_decremental/statistics/'
fd_data = np.loadtxt(os.path.join(path_to_fd,mean_path),delimiter=',')

path_to_freeze = '../runs_trainings/from_checkpoint_sgd/freeze/incremental_decremental/statistics/'
freeze_data = np.loadtxt(os.path.join(path_to_freeze,mean_path),delimiter=',')


In [18]:
# ap incdec
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
pre_trained_path = os.path.join(output_path,'pre_trained')
if not os.path.exists(pre_trained_path):
    os.mkdir(pre_trained_path)
ouput_comparison_path = os.path.join(pre_trained_path,'comparison')
if not os.path.exists(ouput_comparison_path):
    os.mkdir(ouput_comparison_path)
ouput_ap_path = os.path.join(ouput_comparison_path,'ap')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
incdec_path = os.path.join(ouput_ap_path,'incdec')
if not os.path.exists(incdec_path):
    os.mkdir(incdec_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    nofreeze_list = []
    freeze_list = []
    lwf_list = []
    fd_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        nofreeze_list.append(nofreeze_data[pos_subcat_in_data])
        freeze_list.append(freeze_data[pos_subcat_in_data])
        lwf_list.append(lwf_data[pos_subcat_in_data])
        fd_list.append(fd_data[pos_subcat_in_data])
        
        

    subcat_means = {
        'FineTune': [nofreeze_list],
        'FeatExtr': [freeze_list],
        'LWF': [lwf_list],
        'FD': [fd_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat AP')
    ax.set_title('Subcategories Forgetting AP - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(incdec_path + '/class_' + class_name + '.png')
    plt.close(fig)

In [19]:
# Comparison between best results
# ap of subcategories for decremental
mean_path = 'mean_over_tasks/forg_ap_per_subcategory.out'
std_path = 'std_over_tasks/forg_ap_per_subcategory.out'


path_to_lwf = '../runs_trainings/from_checkpoint_sgd/lwf/decremental/statistics/'
lwf_data = np.loadtxt(os.path.join(path_to_lwf,mean_path),delimiter=',')

path_to_nofreeze = '../runs_trainings/from_checkpoint_sgd/no_freeze/decremental/statistics/'
nofreeze_data = np.loadtxt(os.path.join(path_to_nofreeze,mean_path),delimiter=',')

path_to_fd = '../runs_trainings/from_checkpoint_sgd/fd/decremental/statistics/'
fd_data = np.loadtxt(os.path.join(path_to_fd,mean_path),delimiter=',')

path_to_freeze = '../runs_trainings/from_checkpoint_sgd/freeze/decremental/statistics/'
freeze_data = np.loadtxt(os.path.join(path_to_freeze,mean_path),delimiter=',')


In [20]:
# ap incdec
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
pre_trained_path = os.path.join(output_path,'pre_trained')
if not os.path.exists(pre_trained_path):
    os.mkdir(pre_trained_path)
ouput_comparison_path = os.path.join(pre_trained_path,'comparison')
if not os.path.exists(ouput_comparison_path):
    os.mkdir(ouput_comparison_path)
ouput_ap_path = os.path.join(ouput_comparison_path,'ap')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
incdec_path = os.path.join(ouput_ap_path,'decremental')
if not os.path.exists(incdec_path):
    os.mkdir(incdec_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    nofreeze_list = []
    freeze_list = []
    lwf_list = []
    fd_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        nofreeze_list.append(nofreeze_data[pos_subcat_in_data])
        freeze_list.append(freeze_data[pos_subcat_in_data])
        lwf_list.append(lwf_data[pos_subcat_in_data])
        fd_list.append(fd_data[pos_subcat_in_data])
        
        

    subcat_means = {
        'FineTune': [nofreeze_list],
        'FeatExtr': [freeze_list],
        'LWF': [lwf_list],
        'FD': [fd_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat AP')
    ax.set_title('Subcategories Forgetting AP - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(incdec_path + '/class_' + class_name + '.png')
    plt.close(fig)

In [21]:
# Comparison between best results
# ap of subcategories for joint_incremental
mean_path = 'mean_over_tasks/forg_ap_per_subcategory.out'
std_path = 'std_over_tasks/forg_ap_per_subcategory.out'


path_to_lwf = '../runs_trainings/from_checkpoint_sgd/lwf/joint_incremental/statistics/'
lwf_data = np.loadtxt(os.path.join(path_to_lwf,mean_path),delimiter=',')

path_to_nofreeze = '../runs_trainings/from_checkpoint_sgd/no_freeze/joint_incremental/statistics/'
nofreeze_data = np.loadtxt(os.path.join(path_to_nofreeze,mean_path),delimiter=',')

path_to_fd = '../runs_trainings/from_checkpoint_sgd/fd/joint_incremental/statistics/'
fd_data = np.loadtxt(os.path.join(path_to_fd,mean_path),delimiter=',')

path_to_freeze = '../runs_trainings/from_checkpoint_sgd/freeze/joint_incremental/statistics/'
freeze_data = np.loadtxt(os.path.join(path_to_freeze,mean_path),delimiter=',')

In [22]:
# ap incdec
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
pre_trained_path = os.path.join(output_path,'pre_trained')
if not os.path.exists(pre_trained_path):
    os.mkdir(pre_trained_path)
ouput_comparison_path = os.path.join(pre_trained_path,'comparison')
if not os.path.exists(ouput_comparison_path):
    os.mkdir(ouput_comparison_path)
ouput_ap_path = os.path.join(ouput_comparison_path,'ap')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
incdec_path = os.path.join(ouput_ap_path,'joint_incremental')
if not os.path.exists(incdec_path):
    os.mkdir(incdec_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    nofreeze_list = []
    freeze_list = []
    lwf_list = []
    fd_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        nofreeze_list.append(nofreeze_data[pos_subcat_in_data])
        freeze_list.append(freeze_data[pos_subcat_in_data])
        lwf_list.append(lwf_data[pos_subcat_in_data])
        fd_list.append(fd_data[pos_subcat_in_data])
        

        

    subcat_means = {
        'FineTune': [nofreeze_list],
        'FeatExtr': [freeze_list],
        'LWF': [lwf_list],
        'FD': [fd_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat AP')
    ax.set_title('Subcategories Forgetting AP - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(incdec_path + '/class_' + class_name + '.png')
    plt.close(fig)

In [23]:
# Comparison between best results
# ap of subcategories for baseline
mean_path = 'mean_over_tasks/forg_ap_per_subcategory.out'
std_path = 'std_over_tasks/forg_ap_per_subcategory.out'


path_to_lwf = '../runs_trainings/from_checkpoint_sgd/lwf/data_substitution/statistics/'
lwf_data = np.loadtxt(os.path.join(path_to_lwf,mean_path),delimiter=',')

path_to_nofreeze = '../runs_trainings/from_checkpoint_sgd/no_freeze/data_substitution/statistics/'
nofreeze_data = np.loadtxt(os.path.join(path_to_nofreeze,mean_path),delimiter=',')

path_to_fd = '../runs_trainings/from_checkpoint_sgd/fd/data_substitution/statistics/'
fd_data = np.loadtxt(os.path.join(path_to_fd,mean_path),delimiter=',')

path_to_freeze = '../runs_trainings/from_checkpoint_sgd/freeze/data_substitution/statistics/'
freeze_data = np.loadtxt(os.path.join(path_to_freeze,mean_path),delimiter=',')

In [24]:
# ap baseline
output_path = '../statistics_to_save/subcategories_plots/'
if not os.path.exists(output_path):
    os.mkdir(output_path)
pre_trained_path = os.path.join(output_path,'pre_trained')
if not os.path.exists(pre_trained_path):
    os.mkdir(pre_trained_path)
ouput_comparison_path = os.path.join(pre_trained_path,'comparison')
if not os.path.exists(ouput_comparison_path):
    os.mkdir(ouput_comparison_path)
ouput_ap_path = os.path.join(ouput_comparison_path,'ap')
if not os.path.exists(ouput_ap_path):
    os.mkdir(ouput_ap_path)
incdec_path = os.path.join(ouput_ap_path,'baseline')
if not os.path.exists(incdec_path):
    os.mkdir(incdec_path)

for class_name in all_behaviors_dict:
    subcat_names = all_behaviors_dict[class_name]

    nofreeze_list = []
    freeze_list = []
    lwf_list = []
    fd_list = []
    for idx_subcat in range(len(subcat_names)):
        pos_subcat_in_data = subcategories_names.index(subcat_names[idx_subcat])
        nofreeze_list.append(nofreeze_data[pos_subcat_in_data])
        freeze_list.append(freeze_data[pos_subcat_in_data])
        lwf_list.append(lwf_data[pos_subcat_in_data])
        fd_list.append(fd_data[pos_subcat_in_data])
        


        

    subcat_means = {
        'FineTune': [nofreeze_list],
        'FeatExtr': [freeze_list],
        'LWF': [lwf_list],
        'FD': [fd_list],
        }

    x = np.arange(len(subcat_names))  # the label locations
    width = 0.20  # the width of the bars
    multiplier = 0

    fig, ax = plt.subplots(layout='constrained')

    for attribute, measurements in subcat_means.items():
        offset = width * multiplier
        rects = ax.bar(x + offset, measurements[0], width, label=attribute)
        ax.errorbar(x+offset, measurements[0], fmt='.', color='Black', elinewidth=1,capthick=5,errorevery=1, alpha=0.5, ms=4, capsize = 2)
        #ax.bar_label(rects, padding=3)
        multiplier += 1

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Forgetting on subcat AP')
    ax.set_title('Subcategories Forgetting AP - {}'.format(class_name))
    ax.set_xticks(x + width, subcat_names)
    ax.legend(loc='upper left', ncols=4)
    #ax.set_ylim(-5, 15)
    ax.tick_params(axis='x', labelrotation=90)
    ax.axhline(linewidth=1, color='black')
    fig.savefig(incdec_path + '/class_' + class_name + '.png')
    plt.close(fig)